In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

#Hyperparameters

In [ ]:
#PARAMETERS ARE WRONG

In [ ]:
#Transformer block
vocab_size = 100 #len(sp) #No. words in sentencepiece
embed_dim = 128 # Hidden embedding dimension
hidden_dim = 256 # Hidden layer size in feed forward network NEEDS TO BE DIVISIBLE BY NUMBER OF HEADS
num_heads = 4 # Number of attention heads
num_layers = 1  # Number of stacked decoder blocks NIL
dropout = 0.1 #self-explanatory

print(vocab_size)

100


In [ ]:
batch_size = 16 # Batch size for training
# max_seq_len = 256 # Maximum sequence length
lr = 1e-4 # Optimizer learning rate
# beta1 = 0.9 # Adam beta1
# beta2 = 0.999 # Adam beta2
# epsilon = 1e-8 # Adam epsilon

In [ ]:
# iter = 100 #print loss after x batches
num_epochs = 1

In [ ]:
ds_samples = 1000

#Imports and seed stuff


In [ ]:
!pip install sentencepiece
!pip install datasets
!pip install keras-preprocessing
!pip install wandb

In [ ]:
import random
import torch
import math
import sentencepiece as spm
from datasets import load_dataset
import string
import time
import wandb
import datetime
import os

In [ ]:
dataset = load_dataset("roneneldan/TinyStories")

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
torch.manual_seed(42)

#Tokenisation

In [ ]:
#Extract samples from tinystories to train tokeniser
sample_texts = dataset["train"]['text'][:1000]
# take 1000 samples from training set


for i, text in enumerate(sample_texts):
  text = text.lower().strip() # lowercase and remove whitespace
  text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
  sample_texts[i] = text


with open('sentences.txt', 'w') as f:
  for text in sample_texts:
    f.write(text + '\n')
# write samples to sentences.txt

In [ ]:
with open('sentences.txt') as f:
    num_lines = sum(1 for line in f)
print(num_lines)

10426


In [ ]:
spm.SentencePieceTrainer.Train(f"--input=sentences.txt --model_prefix=sp --vocab_size=100 --model_type=unigram")

In [ ]:
import sentencepiece as spm

class TinyStoriesTokenizer:

    def __init__(self, model_path):
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(model_path)

    def encode(self, texts):
        return self.sp.encode(texts, out_type=int)

    def decode(self, ids):
        return self.sp.decode_ids(ids)

    def tokenize(self, text):
        return self.sp.encode_as_pieces(text)

    @property
    def vocab_size(self):
        return self.sp.get_piece_size()

tokenizer = TinyStoriesTokenizer('sp.model')

In [ ]:
import shutil
shutil.copy('sp.model', 'tinystories_tokenizer.model')

'tinystories_tokenizer.model'

In [ ]:
# sp = spm.SentencePieceProcessor()
# sp.load('tinystories_tokenizer.model')

# tokenizer = TinyStoriesTokenizer(sp)

#Dataset handling

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load full dataset
full_df = pd.DataFrame.from_dict(dataset['train'])

# Take 100k subsample
df = full_df.sample(ds_samples, random_state=42)

# Split data into train, validation, test
train, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train, test_size=0.2)

# Set batch size
batch_size = 100

# Create tokenizer
tokenizer = TinyStoriesTokenizer('tinystories_tokenizer.model')

# Lists to store tokenized batches
train_tokens = []
val_tokens = []
test_tokens = []

# Function to iterate through batches
def iterate_batches(df, size):
  for i in range(0, len(df), size):
    yield df[i:i+size]

# Train batches
for batch in iterate_batches(train, batch_size):

  texts = batch['text'].values.tolist()

  tokens = tokenizer.encode(texts)

  train_tokens.append(tokens)


# Validation batches
for batch in iterate_batches(val, batch_size):

  texts = batch['text'].values.tolist()

  tokens = tokenizer.encode(texts)

  val_tokens.append(tokens)


# Test batches
for batch in iterate_batches(test, batch_size):

  texts = batch['text'].values.tolist()

  tokens = tokenizer.encode(texts)

  test_tokens.append(tokens)

In [ ]:
print(train_tokens[0][:10])
print(len(train_tokens[0]))

[[3, 0, 5, 15, 18, 3, 0, 9, 7, 3, 33, 4, 76, 10, 0, 3, 0, 19, 4, 17, 73, 21, 67, 55, 21, 17, 10, 0, 3, 0, 11, 4, 65, 0, 3, 0, 5, 15, 59, 10, 20, 3, 11, 4, 32, 21, 17, 0, 3, 0, 6, 3, 9, 10, 20, 52, 22, 11, 4, 6, 0, 3, 0, 52, 22, 11, 4, 6, 81, 23, 6, 9, 27, 25, 21, 49, 15, 4, 3, 6, 19, 38, 10, 0, 73, 3, 15, 4, 6, 7, 12, 0, 3, 0, 5, 5, 25, 0, 3, 0, 9, 7, 0, 3, 0, 59, 40, 20, 52, 22, 11, 4, 6, 0, 3, 0, 5, 15, 91, 0, 3, 0, 4, 23, 19, 5, 32, 10, 3, 0, 9, 7, 56, 52, 22, 11, 4, 6, 0, 3, 0, 4, 42, 16, 6, 10, 37, 3, 35, 20, 23, 26, 5, 35, 0, 3, 0, 19, 4, 52, 22, 11, 4, 6, 23, 6, 9, 27, 25, 10, 21, 13, 23, 26, 5, 35, 0, 3, 0, 5, 32, 0, 61, 3, 9, 10, 39, 5, 5, 12, 0, 3, 0, 9, 7, 91, 0, 3, 0, 19, 4, 28, 7, 44, 10, 21, 3, 6, 8, 17, 13, 52, 22, 11, 4, 6, 0, 3, 0, 19, 4, 3, 6, 7, 25, 4, 10, 37, 93, 3, 0, 5, 15, 0, 3, 0, 19, 4, 42, 16, 6, 10, 37, 3, 35, 20, 39, 5, 9, 11, 0, 3, 0, 19, 4, 52, 22, 11, 4, 6, 23, 6, 9, 27, 25, 10, 21, 13, 39, 5, 9, 11, 0, 3, 0, 7, 11, 3, 0, 59, 40, 37, 88, 0, 3, 0, 5, 15, 2

In [ ]:
# print(train.head())
# print(val.head())
# print(test.head())

In [ ]:
!pip show keras-preprocessing

Name: Keras-Preprocessing
Version: 1.1.2
Summary: Easy data preprocessing and data augmentation for deep learning models
Home-page: https://github.com/keras-team/keras-preprocessing
Author: Keras Team
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, six
Required-by: 


In [ ]:
from keras_preprocessing.sequence import pad_sequences

In [ ]:
from torch.utils.data import TensorDataset
# Get max sequence length
max_len = max(len(t) for t in train_tokens + val_tokens) + 1

all_train_tokens = [token for batch in train_tokens for token in batch]
all_val_tokens = [token for batch in val_tokens for token in batch]

del train_tokens, val_tokens #ram

# Pad sequences
pad_train_tokens = pad_sequences(all_train_tokens, maxlen=max_len)
pad_val_tokens = pad_sequences(all_val_tokens, maxlen=max_len)

input_seqs = pad_train_tokens[:, :-1]
target_seqs = pad_train_tokens[:, 1:]

del pad_train_tokens #rem

# Convert to tensors
input_tensors = torch.tensor(input_seqs)
target_tensors = torch.tensor(target_seqs)
train_data = TensorDataset(input_tensors, target_tensors)
val_data = TensorDataset(torch.tensor(pad_val_tokens))
print(train_data[0][0].shape)
del input_seqs, target_seqs, input_tensors, target_tensors, pad_val_tokens

torch.Size([99])


In [ ]:
print(train_data[10])

(tensor([66,  3,  4, 60, 35,  4, 21, 45, 23,  7, 30,  4,  0,  3,  0,  7, 14, 12,
        17,  0, 13,  3, 30,  9,  8,  4, 23, 26,  8,  4,  7, 14, 18, 52, 11, 17,
        42,  4,  5, 26, 12,  4,  3, 12,  5, 10,  6, 13,  9,  8,  3, 19,  5, 15,
         4, 10,  0,  3,  0,  9, 15, 15, 17, 18, 56, 62, 15, 17, 80, 23,  7, 14,
         0, 64, 41,  3, 25, 11,  4, 32, 41, 69,  3,  4,  7, 54,  3,  5,  6, 19,
        46, 21,  3, 12,  4,  7, 11,  3, 35], dtype=torch.int32), tensor([ 3,  4, 60, 35,  4, 21, 45, 23,  7, 30,  4,  0,  3,  0,  7, 14, 12, 17,
         0, 13,  3, 30,  9,  8,  4, 23, 26,  8,  4,  7, 14, 18, 52, 11, 17, 42,
         4,  5, 26, 12,  4,  3, 12,  5, 10,  6, 13,  9,  8,  3, 19,  5, 15,  4,
        10,  0,  3,  0,  9, 15, 15, 17, 18, 56, 62, 15, 17, 80, 23,  7, 14,  0,
        64, 41,  3, 25, 11,  4, 32, 41, 69,  3,  4,  7, 54,  3,  5,  6, 19, 46,
        21,  3, 12,  4,  7, 11,  3, 35,  0], dtype=torch.int32))


In [ ]:
from torch.utils.data import DataLoader

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)

#Transformer model

In [ ]:
import torch
import torch.nn as nn

class ScaledDotProductAttention(nn.Module):
    '''
    Scaled dot-product attention mechanism.
    '''

    def __init__(self, dropout):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, scale=None, attn_mask=None):
        print(query.shape, key.shape, value.shape)
        # Scale dot product scores
        if scale is None:
            scale = 1 / (key.size(-1) ** 0.5)
        scores = torch.matmul(query, key.transpose(-2, -1)) * scale
        print("Scores shape:", scores.shape)

        # Apply attention mask
        if attn_mask is not None:
            scores = scores.masked_fill(attn_mask==0, -1e9)
            print("Scores shape after mask:", scores.shape)

        # Normalize with softmax
        attn_probs = nn.Softmax(dim=-1)(scores)

        # Apply dropout
        attn_probs = self.dropout(attn_probs)

        # Multiply with value vectors
        context = torch.matmul(attn_probs, value)

        return context

In [ ]:
class FeedForwardNetwork(nn.Module):

    def __init__(self, embed_dim, hidden_dim, dropout):

        super().__init__()

        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim

        self.linear1 = nn.Linear(embed_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, embed_dim)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.linear2(x)
        x = self.dropout(x)

        return x

In [ ]:
class MultiHeadedAttention(nn.Module):

    def __init__(self, embed_dim, num_heads, dropout):
        super().__init__()

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)

        self.attention = ScaledDotProductAttention(dropout)

        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, attn_mask):

        # Retain shape after projection
        query = self.q_proj(query)
        key = self.k_proj(key)
        value = self.v_proj(value)

        # Split heads by slicing
        head_dim = self.embed_dim // self.num_heads
        q_slices = [query[:,:,i*head_dim:(i+1)*head_dim] for i in range(self.num_heads)]
        k_slices = [key[:,:,i*head_dim:(i+1)*head_dim] for i in range(self.num_heads)]
        v_slices = [value[:,:,i*head_dim:(i+1)*head_dim] for i in range(self.num_heads)]
        print(q_slices[0].shape)

        # Attention
        scale = self.head_dim ** -0.5
        outputs = [self.attention(q, k, v, scale, attn_mask) for q, k, v in zip(q_slices, k_slices, v_slices)]

        # Recombine heads
        output = torch.cat(outputs, dim=-1)

        attn_out = self.out_proj(output)

        return attn_out

In [ ]:
# class MultiHeadedAttention(nn.Module):

#     def __init__(self, embed_dim, num_heads, dropout):

#         super().__init__()

#         self.embed_dim = embed_dim
#         self.num_heads = num_heads
#         self.head_dim = embed_dim // num_heads

#         self.q_proj = nn.Linear(embed_dim, embed_dim)
#         self.k_proj = nn.Linear(embed_dim, embed_dim)
#         self.v_proj = nn.Linear(embed_dim, embed_dim)

#         self.attention = ScaledDotProductAttention(dropout)

#         self.out_proj = nn.Linear(embed_dim, embed_dim)
#         self.dropout = nn.Dropout(dropout)

    # def forward(self, query, key, value, attn_mask):

    #     # Split into heads
    #     batch_size, seq_len, embed_dim = query.size()

    #     query = self.q_proj(query).view(batch_size, seq_len, self.num_heads, self.head_dim)
    #     key = self.k_proj(key).view(batch_size, seq_len, self.num_heads, self.head_dim)
    #     value = self.v_proj(value).view(batch_size, seq_len, self.num_heads, self.head_dim)
    # #     print("q, k, v:", query.shape, key.shape, value.shape)
    #     # Scaled dot product attention
    #     scale = self.head_dim ** -0.5
    #     attn_weights = self.attention(query, key, value, scale, attn_mask)

    #     # Recombine heads
    #     attn_out = attn_weights.transpose(1, 2).reshape(batch_size, seq_len, embed_dim)

    #     # Final linear projection
    #     attn_out = self.out_proj(attn_out)

    #     return attn_out


In [ ]:
import torch
import torch.nn as nn

class DecoderBlock(nn.Module):

    def __init__(self, embed_dim, hidden_dim, num_heads, dropout):
       super().__init__()

       self.embed_dim = embed_dim
       self.hidden_dim = hidden_dim
       self.num_heads = num_heads
       self.dropout = dropout

       self.attn = MultiHeadedAttention(embed_dim, num_heads, dropout)
       self.ffn = FeedForwardNetwork(embed_dim, hidden_dim, dropout)

       self.norm1 = nn.LayerNorm(embed_dim)
       self.norm2 = nn.LayerNorm(embed_dim)

       self.drop = nn.Dropout(dropout)


    def forward(self, x, mask):

      attn_out = self.attn(query=x, key=x, value=x, attn_mask=mask)
      x = x + self.drop(self.norm1(attn_out))

      ffn_out = self.ffn(x)
      x = x + self.drop(self.norm2(ffn_out))

      return x

In [ ]:
class Transformer(nn.Module):

    def __init__(self, vocab_size, embed_dim, hidden_dim, dropout, num_layers, num_heads):

        super().__init__()

        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers  # Add the num_layers hyperparameter
        self.num_heads = num_heads
        self.dropout = dropout


        self.embedding = nn.Embedding(vocab_size, embed_dim)

        self.decoders = nn.ModuleList([DecoderBlock(embed_dim, hidden_dim, num_heads, dropout) for _ in range(num_layers)])

        self.layer_norm = nn.LayerNorm(embed_dim)

        self.out_proj = nn.Linear(embed_dim, vocab_size)


    def get_pos_matrix(self, x): #can be improved/see learned pos encoding & learned angle divisor param (instead of 10000)
        print(x.shape)
        batch_size, sequence_length = x.shape
        store = torch.zeros((batch_size, sequence_length, self.embed_dim)).to(x.device)
        for pos in range(sequence_length):
            for i in range(0, self.embed_dim, 2):
                denominator = 10000 ** (i / self.embed_dim)
                angles = torch.tensor([pos / denominator])
                store[:, pos, i] = torch.sin(angles)
                if i + 1 < self.embed_dim:
                    store[:, pos, i + 1] = torch.cos(angles)
        return store

    def forward(self, x):

        x = self.embedding(x) + self.get_pos_matrix(x)
        print("x shape:", x.shape)
        sequence_length = x.shape[1]
        # Create lower triangular mask
        mask = torch.tril(torch.ones(batch_size, sequence_length, sequence_length)).to(x.device)
        print("Mask shape:", mask.shape)
        for decoder in self.decoders:
            x = decoder(x, mask)

        x = self.layer_norm(x)

        out = self.out_proj(x)

        return out

#debugging

In [ ]:
# import torch

# # Model hyperparameters
# vocab_size = 1000
# embed_dim = 512
# max_seq_len = 64
# num_layers = 6
# num_heads = 8

# # Create dummy input
# dummy_input = torch.rand(1, max_seq_len, embed_dim).long()

# # Initialize model
# model = Transformer(vocab_size, embed_dim, hidden_dim, dropout, num_layers, num_heads)

# # Forward pass on dummy input
# outputs = model(dummy_input)

# # Print output shape
# print(outputs.shape)

# # Print attention scores shape for each layer
# for i, attn in enumerate(model.attentions):
#     print(f"Layer {i} attention scores shape: {attn.shape}")

#Running

In [ ]:
m = Transformer(vocab_size, embed_dim, hidden_dim, dropout, num_layers, num_heads)
opt = torch.optim.Adam(m.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
print("Parameters: ", m.parameters())

Parameters:  <generator object Module.parameters at 0x7a6a0dd11e70>


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
m.to(device)

Transformer(
  (embedding): Embedding(100, 128)
  (decoders): ModuleList(
    (0): DecoderBlock(
      (attn): MultiHeadedAttention(
        (q_proj): Linear(in_features=128, out_features=128, bias=True)
        (k_proj): Linear(in_features=128, out_features=128, bias=True)
        (v_proj): Linear(in_features=128, out_features=128, bias=True)
        (attention): ScaledDotProductAttention(
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (out_proj): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffn): FeedForwardNetwork(
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (relu): ReLU()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (drop): Dropou

In [ ]:
models_dir = 'models'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_path = f"models/{timestamp}_oddformer.pth"

In [ ]:
# Initialise wandb
wandb.init(
  project="oddformer",
  name= "multiheaded_oddformer, 1 epoch",
  config={
  "dataset": "sentences.txt",
  "epochs": num_epochs,
  "batch_size": batch_size,
  }
)

Problem at: <ipython-input-95-27f7f6f5ceb0> 2 <cell line: 2>


CommError: ignored

In [ ]:
wandb.config.update({
  "epochs": num_epochs,
  "batch_size": batch_size,
  "learning_rate": lr
})

In [ ]:
for epoch in range(num_epochs):

  # Training loop
  for X, y in train_loader:
    X, y = X.to(device), y.to(device)
    print(next(m.parameters()).device)
    print(X.device, y.device)
    outputs = m(X)
    loss = criterion(outputs, y)

    loss.backward()
    opt.step()

    wandb.log({"Batch Loss": loss.item()})

  # Validation loop
  val_loss = 0
  val_accuracy = 0

  for X, y in val_loader:
    X, y = X.to(device), y.to(device)
    outputs = m(X)
    loss = criterion(outputs, y)

    val_loss += loss.item()
    val_accuracy += (outputs.argmax(1) == y).sum().item() / len(y)

  val_loss /= len(val_loader)

  print(f"Epoch {epoch} Validation Loss: {val_loss}")
  print(f"Epoch {epoch} Validation Accuracy: {val_accuracy}")

  wandb.log({"Val Loss": val_loss, "Val Accuracy": val_accuracy})

cuda:0
cuda:0 cuda:0
torch.Size([100, 99])
x shape: torch.Size([100, 99, 128])
Mask shape: torch.Size([100, 99, 99])
torch.Size([100, 99, 32])
torch.Size([100, 99, 32]) torch.Size([100, 99, 32]) torch.Size([100, 99, 32])
Scores shape: torch.Size([100, 99, 99])
Scores shape after mask: torch.Size([100, 99, 99])
torch.Size([100, 99, 32]) torch.Size([100, 99, 32]) torch.Size([100, 99, 32])
Scores shape: torch.Size([100, 99, 99])
Scores shape after mask: torch.Size([100, 99, 99])
torch.Size([100, 99, 32]) torch.Size([100, 99, 32]) torch.Size([100, 99, 32])
Scores shape: torch.Size([100, 99, 99])
Scores shape after mask: torch.Size([100, 99, 99])
torch.Size([100, 99, 32]) torch.Size([100, 99, 32]) torch.Size([100, 99, 32])
Scores shape: torch.Size([100, 99, 99])
Scores shape after mask: torch.Size([100, 99, 99])


RuntimeError: ignored

In [ ]:
torch.save(m.state_dict(), "weights.pth")
wandb.save("weights.pth")

wandb.finish()